<a href="https://colab.research.google.com/github/SayNoToMustache/sber_hw_bmla/blob/main/%D0%9B%D0%B8%D1%82%D0%B2%D0%B8%D0%BD%D0%BE%D0%B2_%223_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D0%90%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D0%B8_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%BE%D0%B2%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## О задании
В этом задании вы будете прогнозировать отток клиентов банка с помощью ансамблевых методов. Работать будем с датасетом [Churn for Bank Customers](https://www.kaggle.com/datasets/mathchi/churn-for-bank-customers)
(подробнее с описанием признаков можете ознакомиться по ссылке). Целевая переменная - `Exited` (1 - клиент покинул банк, 0 - клиент не покинул банк)

### Формат сдачи
Данное задание сдаётся через [эту гугл форму](https://forms.gle/jrdU8j2zH6KQ8BATA). Не забудьте открыть доступ по ссылке к файлу (справа сверху "Поделиться" или "Share")

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Загрузим данные. 

Данные можно найти по [ссылке](https://drive.google.com/file/d/1rz00kErCbYlRh5Y0zdYENN8nlixAyLlh/view?usp=share_link)

In [2]:
!gdown --id 1rz00kErCbYlRh5Y0zdYENN8nlixAyLlh

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1rz00kErCbYlRh5Y0zdYENN8nlixAyLlh
To: /content/churn.csv
100% 685k/685k [00:00<00:00, 97.9MB/s]


In [3]:
df = pd.read_csv('churn.csv')

In [4]:
df.shape

(10000, 14)

In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Выведите сводные характеристики данных, воспользовавшись методом `.describe()`.

In [6]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


**Вопрос.** Какая задача машинного обучения решается?

<u>**Ответ**</u>: Обучение с учителем (классификация)

Посмотрите на соотношение классов в целевой переменной `Exited` (выведите, сколько элементов каждого класса или в каком они соотношении)

In [7]:
# print("Кол-во NaN =", df.isna().sum())
# No one nan
df.groupby("Exited").agg(["mean", "count"])

RowNumber          CustomerId       CreditScore              Age  \
               mean count          mean count        mean count       mean   
Exited                                                                       
0       5024.694964  7963  1.569117e+07  7963  651.853196  7963  37.408389   
1       4905.917526  2037  1.569005e+07  2037  645.351497  2037  44.837997   

                Tenure             Balance       NumOfProducts        \
       count      mean count          mean count          mean count   
Exited                                                                 
0       7963  5.033279  7963  72745.296779  7963      1.544267  7963   
1       2037  4.932744  2037  91108.539337  2037      1.475209  2037   

       HasCrCard       IsActiveMember       EstimatedSalary        
            mean count           mean count            mean count  
Exited                                                             
0       0.707146  7963       0.554565  7963    99738.391772  7963  
1       0.699067  2037       0.360825  2037   101465.677531  2037

**Вопрос.** Присутствует ли дисбаланс классов?

<u>**Ответ**</u>: Да, и очень большой. 7963 к 2037

**Вопрос.** Если есть дисбаланс классов, то как с ним справляются? Приведите способы, как обходиться с ним

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

**Вопрос.** Какая метрика важнее в данной задаче: precision или recall? Обоснуйте свой выбор

<u>**Ответ**</u>: precision — это доля релевантных экземпляров среди извлеченных экземпляров, а recall — это доля релевантных экземпляров, которые были извлечены. Таким образом, и точность, и отзыв основаны на релевантности.

Следовательно нам в принципе интересны обе метрики, тк каждая из них говорит о релевантности модели.

Мне кажется что основной будет precission тк он сообщает нам о насколько достоверны результаты. А полнота нас интересует в меньшей степени.

## Отбор признаков для модели

Переведите значения признака `Gender` в числа. В результате должен получиться столбец из 0 и 1

In [8]:
df['Gender'] = df['Gender'].apply(lambda x: 1 if x == 'Male' else 0)

Отберите признаки, на которых вы будете обучать модели. Учтите, что будут обучаться ансамбли из решающих деревьев. Запишите в переменную `X` независимые признаки, а в переменную `y` - целевую переменную. Если необходимо, сделайте кодирование категориальные признаков в числовые.

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Geography_enc'] = le.fit_transform(df['Geography'])
print(le.classes_)
df['Geography_enc'].head()

['France' 'Germany' 'Spain']


0    0
1    2
2    0
3    0
4    2
Name: Geography_enc, dtype: int64

In [10]:
df['EstimatedSalary'].head(10)

0    101348.88
1    112542.58
2    113931.57
3     93826.63
4     79084.10
5    149756.71
6     10062.80
7    119346.88
8     74940.50
9     71725.73
Name: EstimatedSalary, dtype: float64

In [11]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 
feats = ['Age', 'Gender', 'CreditScore', 'Geography_enc', 'Tenure', 'Balance',
         'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'] # сюда добавьте признаки, на основе которых будет строится прогноз

X = df[feats]
y = df['Exited']

## Разбиение данных

Разбейте данные на тренировочные и тестовые

*Не забудьте при разбиении сохранить соотношение классов в тренировочной и тестовой выброках

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

## Моделирование

Мы будем обучать ансамблевые методы. Среди них: случайный лес и градиентный бустинг

**Вопрос.** Чем отличается алгоритм случайного леса и градиентный бустинг?

<u>**Ответ**</u>: При бустинге однородные модели последовательно обучаются, исправляя ошибки друг друга

В то время как случайный лес усредняет показания обученых на разных наборах данных

### Обучение случайного леса

In [14]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score

Обучите модель случайного леса на `X_train`, `y_train`

In [15]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

clf = ExtraTreesClassifier()
clf.fit(X_train, y_train)


ExtraTreesClassifier()

#### Оценка модели случайного леса

Сделайте предсказание обученной модели на `X_test` и оцените precision, recall и f1_score, имея реальные метки `y_test`

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

**Ниже даны значения для RandomForestClassifier**

default

precision = 0.7857142857142857

recall = 0.4594594594594595

f1 = 0.57984496124031

n_estimators = 50

precision = 0.773109243697479

recall = 0.4520884520884521

f1 = 0.5705426356589147

n_estimators = 150 

precision = 0.7777777777777778

recall = 0.44717444717444715

f1 = 0.5678627145085804

In [17]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

y_pred = clf.predict(X_test) # предсказания на X_test
precision = precision_score(y_test, y_pred=y_pred) # значение метрики precision
recall = recall_score(y_test, y_pred=y_pred) # значение метрики recall
f1 = f1_score(y_test, y_pred=y_pred) # значение метрики f1
print('precision =', precision)
print('recall =', recall)
print('f1 =', f1)

from sklearn.metrics import classification_report

report = classification_report(y_test, clf.predict(X_test))
print(report)

precision = 0.726530612244898
recall = 0.43734643734643736
f1 = 0.5460122699386504
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      1593
           1       0.73      0.44      0.55       407

    accuracy                           0.85      2000
   macro avg       0.80      0.70      0.73      2000
weighted avg       0.84      0.85      0.84      2000



**(*) Дополнительно.** Попробуйте варьировать различные гиперпараметры случайного леса (как минимум, число деревьев), а также попробуйте заменить `RandomForestClassifier` на `ExtraTreesClassifier`. Повлияло ли это на метрики? Опишите результаты

<u>**Ответ**</u>: Конечно повлияло, так например 50 деревьев вроде как мало(нужно проверять), 150 вроде как мало. По идее ExtraTreesClassifier должен быть точнее так как основывается на Рандомных деревьях, однако наобум подобрать параметры не вышло.

### Обучение градиентного бустинга

Обучите модель градиентного бустинга на тех же данных

In [18]:
from sklearn.ensemble import GradientBoostingClassifier

In [19]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

clf = GradientBoostingClassifier(n_estimators = 150, learning_rate=0.1)
clf.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=150)

Замерьте те же метрики, что и при случайном лесе

(learning_rate=0.5, n_estimators=150)

precision = 0.7025089605734767

recall = 0.48157248157248156

f1 = 0.5714285714285714

default

precision = 0.7795918367346939

recall = 0.4692874692874693

f1 = 0.5858895705521473

In [20]:
y_pred = clf.predict(X_test) # предсказания на X_test
precision = precision_score(y_test, y_pred=y_pred) # значение метрики precision
recall = recall_score(y_test, y_pred=y_pred) # значение метрики recall
f1 = f1_score(y_test, y_pred=y_pred) # значение метрики f1
print('precision =', precision)
print('recall =', recall)
print('f1 =', f1)

precision = 0.7928286852589641
recall = 0.48894348894348894
f1 = 0.6048632218844985


**(*) Дополнительно.** Попробуйте варьировать различные гиперпараметры градиентного бустинга (как минимум, число деревьев и скорость обучения) Есть ли прирост в качестве? Опишите результаты

<u>**Ответ**</u>: При увеличении кол-ва деревьев полнота увеличивается в то время как percision падает

**Вопрос.** Сравните полученные результаты. Какая модель оказалась лучше по метрике f1?

<u>**Ответ**</u>: Лучшей стала GradientBoostingClassifier(n_estimators=150)

### Перебор гиперпараметров

Выберите модель, у которой вы будете подбирать гиперпараметры

### Я выбрал(-а) <u>**ExtraTreesClassifier**</u>

Перейдите на страницу выбранного алгоритма для нахождения гиперпараметров, которые можно перебирать: [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)/[ExtraTreesClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html), [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

Для подбора гиперпараметров будем использовать [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html), который вместо всех комбинаций гиперпараметров перебирает заданное число случайных комбинаций 

In [21]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

Инициализируйте `RandomizedSearchCV`. Основные моменты:
* Укажите число случайных комбинаций гиперпараметров `n_iter`
* Необходимо добавить в `params` гиперпараметры, которые будут перебираться, а также множество значений их гиперпараметров
* В `RandomizedSearchCV` передайте:
    * экземпляр выбранной модели для классификации
    * `n_iter`
    * стратегию разбиение данных на кросс-валидацию `cv`
    * скоринговую функцию `scoring` (строка 'precision' или 'recall', в зависимости от важности для нас той или иной метрики)

In [38]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

clf_1 = ExtraTreesClassifier() # инициализируйте выбранную модель (случайного леса или градиентного бустинга)

n_iter = 100 # число случайных комбинаций гиперпараметров

params = {
    'n_estimators': [10, 50, 100],
    'criterion' : ["gini", "entropy"],
    'min_samples_split' : [2, 3, 4, 5],
    'min_samples_split' : [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9],
    'max_features' : ["sqrt", "log2"],
    'min_impurity_decrease' : [0.0, 0.1, 0.2, 0.3],
    # 'class_weight' : ["balanced", "balanced_subsample"],
}

skf = StratifiedKFold(5, shuffle=True, random_state=42)

grid = RandomizedSearchCV(
    clf_1,
    param_distributions=params,
    n_iter=n_iter,
    cv=skf,
    scoring='precision_micro', # 'precision' или 'recall'
    # verbose=3
)

Запустите обучение. Не забывайте, что обучаемся на тренировочной выборке

In [39]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

grid.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=ExtraTreesClassifier(), n_iter=100,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_impurity_decrease': [0.0, 0.1, 0.2,
                                                                  0.3],
                                        'min_samples_split': [0.1, 0.2, 0.3,
                                                              0.4, 0.5, 0.7,
                                                              0.9],
                                        'n_estimators': [10, 50, 100]},
                   scoring='precision_micro')

Выведите лучшие найденные гиперпараметры, обратившись к `grid`. Воспользуйтесь атрибутами `best_score_` и `best_params_`

In [40]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

print('Лучший n_estimators = {}'.format(grid.best_estimator_.get_params()['n_estimators']))
print('Лучший criterion = {}'.format(grid.best_estimator_.get_params()['criterion']))
print('Лучший class_weight = {}'.format(grid.best_estimator_.get_params()['class_weight']))
print('Лучший max_features = {}'.format(grid.best_estimator_.get_params()['max_features']))
print('Лучший min_impurity_decrease = {}'.format(grid.best_estimator_.get_params()['min_impurity_decrease']))
print('Лучший min_samples_split = {}'.format(grid.best_estimator_.get_params()['min_samples_split']))
grid.best_score_

Лучший n_estimators = 10
Лучший criterion = gini
Лучший class_weight = None
Лучший max_features = sqrt
Лучший min_impurity_decrease = 0.0
Лучший min_samples_split = 0.1


0.8213750000000001

Сделайте предсказание на тестовой выборке и замерьте метрики

In [41]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

y_pred = grid.predict(X_test)

precision = precision_score(y_test, y_pred=y_pred) # значение метрики precision
recall = recall_score(y_test, y_pred=y_pred) # значение метрики recall
f1 = f1_score(y_test, y_pred=y_pred) # значение метрики f1
print('precision =', precision)
print('recall =', recall)
print('f1 =', f1)

precision = 0.9818181818181818
recall = 0.13267813267813267
f1 = 0.23376623376623376


In [42]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

report = classification_report(y_test, grid.predict(X_test))
cm = confusion_matrix(y_test, grid.predict(X_test))
print(report)
print(cm)

              precision    recall  f1-score   support

           0       0.82      1.00      0.90      1593
           1       0.98      0.13      0.23       407

    accuracy                           0.82      2000
   macro avg       0.90      0.57      0.57      2000
weighted avg       0.85      0.82      0.76      2000

[[1592    1]
 [ 353   54]]


**Вопрос.** Улучшилось ли качество модели после подбора гиперпараметров?

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

Если нет значимых изменений в качестве, то имеет смысл:
* Увеличить число случайных комбинаций `n_iter` (придётся подождать больше времени)
* Добавить другие гиперпараметры
* Расширить множество перебираемых значений гиперпараметров
* Запустить заново `grid.fit` для перебора других случайных комбинаций

### (*) Дополнительно

**(*) Дополнительно.** Возьмите алгоритм `XGBClassifier` из библиотеки `xgboost` и обучите его на данной задаче, подберите гиперпараметры и замерьте метрики. Сравните алгоритм по скорости и полученному качеству

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

**(*) Дополнительно.** Возьмите алгоритм `CatBoostClassifier` из библиотеки `catboost` и обучите его на данной задаче, можете добавить параметр для отображения графиков во время обучения, подберите гиперпараметры и замерьте метрики. Сравните алгоритм по скорости и полученному качеству

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

**(*) Дополнительно.** Возьмите алгоритм стекинга [`StackingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html), обучите его и замерьте качество. В качестве базовых алгоритмов можете брать KNeighborsClassifier, RandomForestClassifier, LogisticRegression, BernoulliNB,.. В качестве мета алгоритма возьмите GradientBoostingClassifier или XGBClassifier. 

Получилось ли обучить его? Какие проблемы возникли? Какое качество в итоге получено?

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

## Выводы

**Вопрос.** Напишите выводы о проделанной работе. Должны содержаться ответы на следующие вопросы:
* Какая модель показала лучшее качество?
* Получилось ли добиться лучшего качества после подбора гиперпараметров?

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _